In [2]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

True

### LLM

In [ ]:
import getpass
if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.2,         # Set temperature (creativity)
    max_tokens=512,          # Set max tokens for response
    top_p=0.95
)

In [5]:
llm.invoke("What is the capital of France?")  # type: ignore

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ded0d14823', 'id': 'chatcmpl-BbM7rhJgOX4D4aTa35eBVa6dt1PSH', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 

### Embeddings Model

In [3]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["OPENAI_API_ENDPOINT"],
    azure_deployment=os.environ["OPENAI_EMBEDDINGS_MODEL_DEPLOYMENT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"]
)

In [4]:
vectors = embeddings.embed_query("what convertors are single led xpe lamps compatible with")  # type: ignore
import json
json.dump(vectors, open("vectors.json", "w"), indent=2)

### Document Parsing and Vector DB Storage

In [6]:
import json
from langchain_community.document_loaders import JSONLoader

file_path = "/Users/sathvika/MCT/SEM-4/industry-project/TAL_Chatbot-1/LangchainAzureChatbot/cosmos_ready.json"

def transform_json_structure(file_path):
    with open(file_path) as f:
        original_data = json.load(f)
    
    # Extract all nested converter objects into a list
    converted_list = [value for key, value in original_data.items()]
    
    return converted_list

# # Transform the JSON structure
# transformed_data = transform_json_structure(file_path)

# # Save to new file if needed
# with open("transformed_converters.json", "w") as f:
#     json.dump(transformed_data, f, indent=2)


In [17]:
loader = JSONLoader(
    file_path="/Users/sathvika/MCT/SEM-4/industry-project/TAL_Chatbot-1/LangchainAzureChatbot/converters_improved.json",
    jq_schema=".[]",
    text_content=False
)
documents = loader.load()
print(f"Loaded {len(documents)} documents")

Loaded 65 documents


In [18]:
docs = []
for doc in documents:
    docs.append(doc.page_content)
print(f"Loaded {len(docs)} documents") # docs for creating embeddings seperately to keep the key:value pairs

Loaded 65 documents


In [19]:
vector_embeddings = embeddings.embed_documents(docs)  # type: ignore
print(f"Generated {len(vector_embeddings)} vector embeddings")
print(vector_embeddings[0])

Generated 65 vector embeddings
[0.0156854260712862, 0.003283464815467596, -0.005623520817607641, -0.01753867045044899, -0.03515791893005371, 0.014893097802996635, -0.014557365328073502, -0.013979905284941196, 0.008097870275378227, -0.0190696120262146, 0.01941877417266369, 0.02649601735174656, -0.03671571612358093, 0.006469567306339741, -0.013543453067541122, 0.03773634508252144, 0.017578959465026855, -0.018559297546744347, 0.009319936856627464, -0.01526911836117506, -0.0029947347939014435, 0.021070577204227448, -0.028523841872811317, -0.006321844644844532, -0.01060243509709835, -0.018908459693193436, 0.010797160677611828, -0.030618812888860703, -0.0033791486639529467, -0.02155403234064579, 0.0017122362041845918, -0.00637891935184598, -0.010871021077036858, -0.03475503996014595, 0.006372204516083002, -0.0013530023861676455, 0.016571760177612305, -0.009528091177344322, 0.00964895449578762, -0.010911309160292149, 0.0033321459777653217, 0.010595721192657948, 0.01431563775986433, -0.0128451

In [20]:
docs[1]

'{"id": "cd021d65-1e28-4e2e-a059-4584d099da1b", "type": "24V DC", "artnr": 40057, "converter_description": "LEDCONVERTER REMOTE 24V 20W IP20 ", "strain_relief": "YES", "location": "INDOOR", "dimmability": "NOT DIMMABLE", "efficiency_full_load": 0.83, "ip": 20, "class": 2, "nom_input_voltage_v": {"min": 195, "max": 254}, "output_voltage_v": {"min": 24, "max": 24}, "barcode": "54 15233 00167 3", "name": "CONVERTER 230V LED 24V 20W INDOOR REMOTE ACC", "listprice": 77, "unit": "PC", "lifecycle": "A", "pdf_link": "https://www.tal.be/sites/default/files/2025-02/40057%20LEDCONVERTER%20230V%20LED%2024V%2020VA.pdf", "lamps": {"m LEDLINE medium power 9,6W": {"min": 2, "max": 2}, "m LEDLINE high power 14,4W": {"min": 1.33, "max": 1.33}, "m LEDLINE very high power 19,2W": {"min": 1, "max": 1}}, "size": "109*50*35"}'

In [4]:
indexing_policy = {
    "indexingMode": "consistent",
    "includedPaths": [{"path": "/*"}],  # Indexes all properties, including nested
    "excludedPaths": [
        {
            "path": '/"_etag"/?'
        },
        {
            "path": "/embedding/*"
        }
        ],
}


vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path": "/embedding",
            "dataType": "float32",
            "distanceFunction": "cosine",
            "dimensions": 1536,
        }
    ]
}

full_text_policy = {
    "defaultLanguage": "en-US",
    "fullTextPaths": [{"path": "/text", "language": "en-US"}],
}

In [5]:
import uuid
from azure.cosmos import CosmosClient, PartitionKey
from langchain_community.vectorstores.azure_cosmos_db_no_sql import (
    AzureCosmosDBNoSqlVectorSearch,
)
from langchain_openai import OpenAIEmbeddings
import json
HOST = os.environ["AZURE_COSMOS_DB_ENDPOINT"]
KEY = os.environ["AZURE_COSMOS_DB_KEY"]

cosmos_client = CosmosClient(HOST, KEY)
database_name = "TAL"
container_name = "Converters"
partition_key = PartitionKey(path="/artnr")
cosmos_container_properties = {"partition_key": partition_key}

database = cosmos_client.get_database_client("TAL")

database = cosmos_client.create_database_if_not_exists(database_name)
container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=partition_key,
    indexing_policy=indexing_policy,
    default_ttl=-1)
# with open('/Users/sathvika/MCT/SEM-4/industry-project/TAL_Chatbot-1/LangchainAzureChatbot/converters_improved.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)
item={}
item["id"] = str(uuid.uuid4())
# item["output_voltage"] = item.get("output_voltage_(v)")
# item["nom_input_voltage"] = item.get("nom._input_voltage_(v)")
# item["efficiency_full_load"] = item.get("efficiency_@full_load")
# item["size"] = item.get("size_l*b*h_(mm)")
# item.pop("output_voltage_(v)", None)
# item.pop("nom._input_voltage_(v)", None)
# item.pop("efficiency_@full_load", None)
item["artnr"]=988888
container.create_item(item)

# for item,embedding in zip(data, vector_embeddings):
#     # print(type(item))
#     item["id"] = str(uuid.uuid4())
#     item["embedding"] = embedding  # type: ignore
#     container.create_item(item)


{'id': 'c66480b6-e766-4fae-9b4e-4dffbcc93eae',
 'artnr': 988888,
 '_rid': 'yTVOALzLAcVCAAAAAAAAAA==',
 '_self': 'dbs/yTVOAA==/colls/yTVOALzLAcU=/docs/yTVOALzLAcVCAAAAAAAAAA==/',
 '_etag': '"3700c147-0000-4700-0000-684abfee0000"',
 '_attachments': 'attachments/',
 '_ts': 1749729262}

### Custom Prompt

In [5]:
from langchain_core.prompts import ChatPromptTemplate

custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """"You are a helpful technical assistant for TAL BV and assist users in finding information."
                "Use the provided documentation to answer questions accurately with details about price, pdf_link, input voltage and output voltage if available. """),
        ("human", """Context: {context}
            
        Question: {question}

        Answer:""")
])

instructions=(
                ),

### Langchain preferred method

In [12]:
vector_store = AzureCosmosDBNoSqlVectorSearch(
    vector_embedding_policy=vector_embedding_policy,
    embedding=embeddings,
    indexing_policy=indexing_policy,
    cosmos_client=cosmos_client,
    database_name="langchain_python_db", # db with full text search
    container_name="langchain_python_container",
    cosmos_container_properties=cosmos_container_properties,
    cosmos_database_properties={},
    full_text_policy=full_text_policy,
    full_text_search_enabled=True)

/var/folders/hy/_8988y6s5_7cq87h_9d57rzm0000gn/T/ipykernel_29027/1164264011.py:1: LangChainDeprecationWarning: The class `AzureCosmosDBNoSqlVectorSearch` was deprecated in LangChain 0.3.22 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-azure-ai package and should be used instead. To use it run `pip install -U :class:`~langchain-azure-ai` and import as `from :class:`~langchain_azure_ai.vectorstores import AzureCosmosDBNoSqlVectorSearch``.
  vector_store = AzureCosmosDBNoSqlVectorSearch(


In [8]:
from langchain import hub
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document


prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = custom_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question": "How many single led xpe lamps can ARTNR 930605 be connceted to?"})
print(response["answer"])

### Chatbot using SQL queries for retrieval

In [26]:
from langchain_core.documents import Document

def retrieve2(state: State):
    # Convert natural language question to vector
    query_vector = embeddings.embed_query(state["question"])
    
    # Execute vector search query
    sql_query = """
    SELECT TOP 5 
        c.id,
        c["CONVERTER DESCRIPTION:"],
        c["ARTNR"],
        c["TYPE"],
        c["lamps"],
        VectorDistance(c.embedding, @vector) AS SimilarityScore
    FROM c 
    ORDER BY VectorDistance(c.embedding, @vector)
    """
    
    parameters = [{"name": "@vector", "value": query_vector}]
    
    # Get raw results from Cosmos DB - tal_DB with the key-value pairs
    results = list(container.query_items(
        query={"query": sql_query, "parameters": parameters},
        enable_cross_partition_query=True
    ))
    
    # Convert to LangChain Documents
    documents = []
    for result in results:
        content = f"""
        Converter {result['ARTNR']}: {result['CONVERTER DESCRIPTION:']}
        Type: {result['TYPE']}
        Lamps: {json.dumps(result['lamps'])}
        Similarity: {result['SimilarityScore']}
        """
        
        metadata = {
            "id": result['id'],
            "artnr": result['ARTNR'],
            "type": result['TYPE']        }
        
        documents.append(Document(
            page_content=content.strip(),
            metadata=metadata
        ))
    
    return {"context": documents}

def generate2(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = custom_prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder2 = StateGraph(State).add_sequence([retrieve2, generate2])
graph_builder2.add_edge(START, "retrieve2")
graph2 = graph_builder2.compile()

In [27]:
response = graph2.invoke({"question": "what convertors are single led xpe lamps compatible with"})
print(response["answer"])

The following converters are compatible with Single LED XPE lamps:

1. **Converter 930573: POWERLED CONVERTER 350mA 8W IP54**
   - Minimum: 1
   - Maximum: 7

2. **Converter 930541: POWERLED CONVERTER 350mA 9W IP20**
   - Minimum: 1
   - Maximum: 7

3. **Converter 930537: POWERLED CONVERTER 350mA 26W IP66**
   - Minimum: 12
   - Maximum: 23

4. **Converter 930560: POWERLED CONVERTER REMOTE 350mA 15W IP20 1-10V**
   - Minimum: 2
   - Maximum: 13

5. **Converter 930536: POWERLED CONVERTER 350mA 26W IP66 1-10V**
   - Minimum: 12
   - Maximum: 23

These converters support varying quantities of Single LED XPE lamps, ranging from 1 to 23 depending on the specific converter.
